In [16]:
import cv2
import os
from os import listdir
from os.path import isfile, join
from random import shuffle
import tensorflow as tf
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# img size is 727 x 294

train_data = './specs/_train'
test_data = './specs/_test'

In [17]:
# Returns genre of audio file if genre name is found in filename
def one_hot_label(filename):
    one_hot = np.array([0, 0, 0, 0, 0, 0])
    genres = ['classical', 'hiphop', 'jazz', 'metal', 'reggae', 'pop']
    for i, genre in enumerate(genres):
        if genre in filename:
            one_hot[i] = 1
            return one_hot
    raise ValueError("One or more files is not labeled with a valid genre")

def train_data_with_label():
    train_images = []
    first = True
    for subdir, dirs, files in os.walk(train_data):
        if first == True:
            first = False
            continue
        for i in tqdm(os.listdir(subdir)):
            path = os.path.join(subdir, i)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (121, 49))
            train_images.append([np.array(img), one_hot_label(i)])
    shuffle(train_images)
    return train_images

def test_data_with_label():
    train_images = []
    for i in tqdm(os.listdir(test_data)):
        path = os.path.join(test_data, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (121, 49))
        train_images.append([np.array(img), one_hot_label(i)])
    return train_images


In [18]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *

training_images = train_data_with_label()
testing_images = test_data_with_label()

tr_img_data = np.array([i[0] for i in training_images]).reshape(-1, 121, 49, 1)
tr_lbl_data = np.array([i[1] for i in training_images])

tst_img_data = np.array([i[0] for i in testing_images]).reshape(-1, 121, 49, 1)
tst_lbl_data = np.array([[1] for i in testing_images])

100%|█████████████████████████████████████████████████████████████████████████████████| 60/60 [00:00<00:00, 324.28it/s]


In [19]:
model = Sequential()

model.add(InputLayer(input_shape=[121,49,1]))
model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters=50, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Conv2D(filters=80, kernel_size=5, strides=1, padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=5, padding='same'))

model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(6, activation='softmax'))
optimizer = Adam(lr=1e-3)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=tr_img_data, y=tr_lbl_data, epochs=50, batch_size=100)
model.summary()

Epoch 1/50
540/540 [==============================] - ETA: 5s - loss: 9.9822 - acc: 0.120 - ETA: 3s - loss: 9.1926 - acc: 0.140 - ETA: 2s - loss: 8.3803 - acc: 0.146 - ETA: 1s - loss: 7.8155 - acc: 0.147 - ETA: 0s - loss: 6.9796 - acc: 0.164 - 5s 8ms/step - loss: 6.7149 - acc: 0.1648
Epoch 2/50
540/540 [==============================] - ETA: 3s - loss: 2.1057 - acc: 0.240 - ETA: 2s - loss: 2.0664 - acc: 0.225 - ETA: 1s - loss: 2.1117 - acc: 0.190 - ETA: 1s - loss: 2.0541 - acc: 0.195 - ETA: 0s - loss: 2.0153 - acc: 0.192 - 4s 8ms/step - loss: 2.0027 - acc: 0.1889
Epoch 3/50
540/540 [==============================] - ETA: 4s - loss: 1.8045 - acc: 0.200 - ETA: 3s - loss: 1.8011 - acc: 0.230 - ETA: 2s - loss: 1.8329 - acc: 0.213 - ETA: 1s - loss: 1.8346 - acc: 0.197 - ETA: 0s - loss: 1.8304 - acc: 0.190 - 5s 9ms/step - loss: 1.8279 - acc: 0.1907
Epoch 4/50
540/540 [==============================] - ETA: 3s - loss: 1.7632 - acc: 0.210 - ETA: 2s - loss: 1.7415 - acc: 0.220 - ETA: 1s - loss:

540/540 [==============================] - ETA: 3s - loss: 0.6499 - acc: 0.760 - ETA: 2s - loss: 0.6441 - acc: 0.760 - ETA: 1s - loss: 0.7245 - acc: 0.736 - ETA: 0s - loss: 0.7440 - acc: 0.730 - ETA: 0s - loss: 0.7429 - acc: 0.734 - 4s 7ms/step - loss: 0.7363 - acc: 0.7352
Epoch 30/50
540/540 [==============================] - ETA: 3s - loss: 0.7499 - acc: 0.670 - ETA: 2s - loss: 0.7741 - acc: 0.650 - ETA: 1s - loss: 0.7611 - acc: 0.676 - ETA: 0s - loss: 0.7444 - acc: 0.682 - ETA: 0s - loss: 0.7212 - acc: 0.702 - 4s 7ms/step - loss: 0.7088 - acc: 0.7074
Epoch 31/50
540/540 [==============================] - ETA: 3s - loss: 0.6795 - acc: 0.730 - ETA: 2s - loss: 0.6079 - acc: 0.765 - ETA: 1s - loss: 0.6269 - acc: 0.766 - ETA: 0s - loss: 0.6511 - acc: 0.747 - ETA: 0s - loss: 0.6396 - acc: 0.752 - 4s 7ms/step - loss: 0.6411 - acc: 0.7556
Epoch 32/50
540/540 [==============================] - ETA: 3s - loss: 0.6857 - acc: 0.740 - ETA: 2s - loss: 0.6199 - acc: 0.765 - ETA: 1s - loss: 0.6810 

In [20]:
genres = ['classical', 'hiphop', 'jazz', 'metal', 'pop', 'reggae']
filenames = [name for name in os.listdir(test_data)]

total_samples = len(filenames)
num_correct = 0
second_correct = 0
third_correct = 0

for cnt, data in enumerate(testing_images):
    img = data[0]
    data = img.reshape(1, 121, 49, 1)
    model_out = model.predict([data])
    
    argsort = np.argsort(model_out[0])
#     print(model_out[0])
#     print(argsort)
    
    first, second, third = argsort[-1], argsort[-2], argsort[-3]

    if genres[first] in filenames[cnt]:
#         print(genres[first], filenames[cnt])
        num_correct += 1
    elif genres[second] in filenames[cnt]:
        second_correct += 1
    elif genres[third] in filenames[cnt]:
        third_correct += 1

    formatted_result = """
    '{predicted}' with {probability} for file {filename}
    --------------------------------------------------------------------
    '{second_genre}' with {prob2} is second most likely
    '{third_genre}' with {prob3} is third most likely
    """.format(predicted=genres[first], probability=model_out[0][first], 
               filename=filenames[cnt], second_genre=genres[second],
              prob2=model_out[0][second], third_genre=genres[third],
              prob3=model_out[0][third])

#     print(genres[np.argmax(model_out[0])])
    print(formatted_result, end='')

accuracy = num_correct/total_samples
top2 = (num_correct + second_correct) / total_samples
top3 = (num_correct + second_correct + third_correct) / total_samples

print("""\n    {num_correct} out of {total_samples} classified correctly with {accuracy}% accuracy
    {top2} chance of genre being in top 2 choices
    {top3} chance of genre being in top 3""".format(num_correct=num_correct, total_samples=total_samples, 
                                                    accuracy=accuracy, top2=top2, top3=top3))


    'jazz' with 0.7429276704788208 for file classical.00000.jpg
    --------------------------------------------------------------------
    'classical' with 0.13132944703102112 is second most likely
    'pop' with 0.1154620498418808 is third most likely
    
    'classical' with 0.8333437442779541 for file classical.00001.jpg
    --------------------------------------------------------------------
    'jazz' with 0.15504781901836395 is second most likely
    'pop' with 0.010228877887129784 is third most likely
    
    'classical' with 0.4405667185783386 for file classical.00002.jpg
    --------------------------------------------------------------------
    'jazz' with 0.31810784339904785 is second most likely
    'pop' with 0.13093359768390656 is third most likely
    
    'jazz' with 0.8587684035301208 for file classical.00003.jpg
    --------------------------------------------------------------------
    'pop' with 0.11784739792346954 is second most likely
    'classical' with 0

    
    'metal' with 0.9954655766487122 for file metal.00002.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.003348605940118432 is second most likely
    'reggae' with 0.0005355774192139506 is third most likely
    
    'metal' with 0.9303457140922546 for file metal.00003.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.06902848184108734 is second most likely
    'pop' with 0.0005357949994504452 is third most likely
    
    'metal' with 0.9071290493011475 for file metal.00004.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.07555992156267166 is second most likely
    'reggae' with 0.008954262360930443 is third most likely
    
    'metal' with 0.9886302947998047 for file metal.00005.jpg
    --------------------------------------------------------------------
    'hiphop' with 0.010488715022802353 is second most likely
    'reggae' with 0.000